In [1]:
import numpy as np

# My idea

We are going to solve the folloeing problem

$$
            \begin{dcases}
                \min_{p \geq 0, \textcolor{red}{\pi \in Y}} \sum_{m=1}^{M}\frac{1}{M}\sum_{r=1}^{R}\sum_{s=1}^{S^{m}} \| \xi_r - \zeta_{s}^{m}\|^2 \pi_{rs}^{m} \\
                \textbf{s.t }\sum_{r=1}^R \pi_{rs}^{(m)} = q_s^{(m)}, \quad 
                \sum_{s=1}^{S^{(m)}} \pi_{rs}^{(m)} = p_r 
            \end{dcases}
$$

With constrains such as: $Y = \{\pi \,\large| \,||\pi^{(m)}||_F \leq \tau\}$ or $\sum_{s} \pi_{rs} \leq c_r$.

So my idea is: As the transport plan are matrices we could solve the following problem:

$$
\begin{dcases}
    \min_{Y} \frac{1}{2} ||P - \pi ||_F^2\\

    \textbf{s.t }\sum_{r=1}^R P_{rs}^{(m)} = q_s^{(m)}, \quad 
    \sum_{s=1}^{S^{(m)}} P_{rs}^{(m)} = p_r
\end{dcases}
$$

That is we could find the projection of $\pi$ on $Y \cap \mathcal{U}(\nu^{m},\mu)$ where $\mathcal{U}(\nu^{m},\mu)$ is the set of couplings betewn $\nu^{m},\mu$.

## ADMM (Converges on the frobenius norm and the couplings politope)

We can reformulate the problem as follows:

$$
\begin{dcases}
    \min_{P,Z} \frac{1}{2} ||P - \pi ||_F^2 + \mathcal{I}_{\mathcal{U}(\nu^{m},\mu)} (P) + \mathcal{I}_{\mathcal{B}_{\tau}}(Z) \\   

    \textbf{s.t } P - Z = 0
\end{dcases}
$$

This leads to tink on the ADMM algo. The augmented lagrangian is:

$$
    \mathcal{L}_{\rho}(P,Z,Y) = \frac{1}{2} ||P - \pi ||_F^2 + \mathcal{I}_{\mathcal{U}(\nu^{m},\mu)} (P) + \mathcal{I}_{\mathcal{B}_{\tau}}(Z) + \langle Y, P-Z \rangle + \frac{\rho}{2}||P-Z||_F^2
$$

To simplify this we see that:

$$
\frac{\rho}{2} ||X + \frac{Y}{\rho}||^2 - \frac{1}{2\rho}||Y||^2 = \frac{\rho}{2}\big( ||X||^2 + 2\langle X, \frac{Y}{\rho} \rangle + ||\frac{Y}{\rho}||^2\big)-\frac{1}{2\rho}||Y||^2\\
=\frac{\rho}{2}||X||^2 + \langle X, Y \rangle 
$$

If $X = P-Z$: 

$$
\langle P-Z, Y \rangle + \frac{\rho}{2}||P-Z||^2 =\frac{\rho}{2} ||P - Z + \frac{Y}{\rho}||^2 -\frac{1}{2}||\frac{Y}{\rho}||^2
$$

If we do $U = \frac{Y}{\rho}$ we get the new lagrangian:

$$
    \mathcal{L}_{\rho}(P,Z,U) = \frac{1}{2} ||P - \pi ||_F^2 + \mathcal{I}_{\mathcal{U}(\nu^{m},\mu)} (P) + \mathcal{I}_{\mathcal{B}_{\tau}}(Z) + \frac{\rho}{2} ||P - Z + U||^2 -\frac{1}{2}||U||^2
$$

So we propoce the following iterations:

$$
P^{k+1} = \argmin_{P \in \mathcal{U}(\nu^{m},\mu)}  \frac{1}{2} ||P - \pi ||_F^2  + \frac{\rho}{2} ||P - (Z^K - U^K)||^2\\

Z^{k+1} = \argmin_{Z \in \mathcal{B}_\tau}  \frac{\rho}{2} ||P^{k+1} - Z + U^k||^2\\

U^{k+1} = U^k + P^{k+1} - Z^{k+1}
$$

We see that, for instance:

$$
Z^{k+1} = \argmin_{Z \in \mathcal{B}_\tau}  \frac{\rho}{2} ||P^{k+1} + U^k  - Z||^2 = \mathcal{P}_{\mathcal{B}_\tau}(P^{k+1} + U^k) = \begin{cases}
                            P^{k+1} + U^k \text{ if } ||P^{k+1} + U^k||_F \leq \tau\\
                            \frac{\tau}{||P^{k+1} + U^k||_F} P^{k+1} + U^k \text{ if } ||P^{k+1} + U^k||_F > \tau        
                        \end{cases}
\\
$$

if we take a look onto the first actualization:

$$
\frac{1}{2} ||P - \pi ||_F^2  + \frac{\rho}{2} ||P - (Z^K - U^K)||^2 = \frac{1}{2}||P||^2 - \langle P,\pi \rangle + \frac{1}{2}||\pi||^2 + \frac{\rho}{2}\big( ||P||^2 - 2\langle P, Z^k - U^k \rangle + ||Z^k - U^k||^2\big)\\
= \frac{1}{2}||P||^2 - \langle P,\pi \rangle + \frac{\rho}{2}||P||^2 - \rho\langle P, Z^k - U^k \rangle\\\
= \frac{1+\rho}{2}||P||^2 - \langle P,\pi + \rho(Z^k - U^k) \rangle
$$

We want to complete the square so:

$$
\frac{1+\rho}{2}||P-M||^2 = \frac{1+\rho}{2}||P||^2 - \langle P,\pi + \rho(Z^k - U^k) \rangle\\
= \frac{1+\rho}{2}||P||^2 - (1+\rho)\langle P,M \rangle + \frac{1+\rho}{2}||M||^2
$$

If $M$ does not depend on $P$ we can ignore it on the minimization:

$$
(1+\rho)\langle P,M \rangle = \langle P,\pi + \rho(Z^k - U^k) \rangle\\
M = \frac{\pi + \rho(Z^k - U^k)}{1+\rho}
$$

So:
$$
P^{k+1} = \argmin_{P \in \mathcal{U}(\nu^{m},\mu)}  \frac{1+\rho}{2}||P-\frac{\pi + \rho(Z^k - U^k)}{1+\rho}||^2 = \mathcal{P}_{\mathcal{U}(\nu^{m},\mu)} \big(\frac{\pi + \rho(Z^k - U^k)}{1+\rho} \big)\\
$$

Finally we have:

$$
P^{k+1} = \mathcal{P}_{\mathcal{U}(\nu^{m},\mu)} \big(\frac{\pi + \rho(Z^k - U^k)}{1+\rho} \big)\\

Z^{k+1} = \begin{cases}
            P^{k+1} + U^k \text{ if } ||P^{k+1} + U^k||_F \leq \tau\\
            \frac{\tau}{||P^{k+1} + U^k||_F} P^{k+1} + U^k \text{ if } ||P^{k+1} + U^k||_F > \tau        
        \end{cases}\\
U^{k+1} = U^k + P^{k+1} - Z^{k+1}

$$

The only problem is solving the update of $P$, lets suppose that projection exists (PROVE), we can use Dikstra algorihm for this as the set $\mathcal{U}(\nu^{m},\mu) = \{P : P1 = \nu\} \cap \{P : P^{T}1 = \mu\} \cap \{P : P \geq 0\}$ each one of the is convex, we will name them $C_1, C_2, C_3$.

We can see (PROVE) the the projection of a matrix $Q \in \mathbb{R}^{m\times n}$ on each of the previous sets are:

$$
\mathcal{P}_{C_1}(Q) = Q + \frac{\nu^m - Q1}{n}1^{T}\\
\mathcal{P}_{C_2}(Q) = Q + 1\frac{\mu - Q^T1^T}{m}\\
\mathcal{P}_{C_3}(Q) = \max\{Q,0\}
$$

So, Dikstra is as follows:

Let $P^0 = A$ (matrix to project), $R_1 = R_2 = R_3 = 0$ and:

$$
Q_1 = P^{K} + R_1^{K}\\
P_1 = \mathcal{P}_{C_1}(Q_1)\\
R_1^{k+1} = Q_1 - P_1\\
Q_2 = P_1 + R_2^{k}\\
P_2 = \mathcal{P}_{C_2}(Q_2)\\
R_2^{k+1} = Q_2 - P_2\\
Q_3 = P_2 + R_3^{k}\\
P^{k+1} = \mathcal{P}_{C_3}(Q_3)\\
R_3^{k+1} = Q_3 - P^{k+1}\\
$$



### Implementation

Lets implement this algo:

In [2]:
def proj_rows(Q, nu):
    correction = (nu - Q.sum(axis=1))[:, None] / Q.shape[1]
    return Q + correction

def proj_cols(Q, mu):
    correction = (mu - Q.sum(axis=0))[None, :] / Q.shape[0]
    return Q + correction

def proj_simplex(Q):
    return np.maximum(Q, 0)

In [3]:
def dykstra_coupling(
    A,
    nu,
    mu,
    max_iter=500,
    tol=1e-8,
    verbose=False
):
    P = A.copy()
    R1 = np.zeros_like(A)
    R2 = np.zeros_like(A)
    R3 = np.zeros_like(A)

    for k in range(max_iter):

        P_old = P.copy()

        Q1 = P + R1
        P1 = proj_rows(Q1, nu)
        R1 = Q1 - P1

        Q2 = P1 + R2
        P2 = proj_cols(Q2, mu)
        R2 = Q2 - P2

        Q3 = P2 + R3
        P = proj_simplex(Q3)
        R3 = Q3 - P

        err = np.linalg.norm(P - P_old, ord='fro') / (np.linalg.norm(P_old, ord='fro') + 1e-12)

        if verbose and k % 50 == 0:
            print(f"Iter {k:4d} | err = {err:.2e}")

        if err < tol:
            if verbose:
                print(f"Converged at {k} iterations.")
            break

    return P


Lets prove it:

In [4]:
Q = np.array([[np.random.uniform(10, 20) for _ in range(4)] for _ in range(3)])

nu = np.array(np.random.dirichlet(np.ones(3)))
mu = np.array(np.random.dirichlet(np.ones(4)))

print("Initial Q:")
print(Q)

print("\nRow sums of Q:", Q.sum(axis=1))
print("Target row sums (nu):", nu)

print("\nColumn sums of Q:", Q.sum(axis=0))
print("Target column sums (mu):", mu)

Initial Q:
[[19.89586324 17.92702513 11.23067502 12.70142359]
 [17.76155577 11.94742612 19.63911264 11.54894467]
 [12.29422204 13.53855468 18.66601178 18.50994024]]

Row sums of Q: [61.75498698 60.8970392  63.00872873]
Target row sums (nu): [0.2959353  0.13146063 0.57260407]

Column sums of Q: [49.95164105 43.41300593 49.53579944 42.76030849]
Target column sums (mu): [0.29222391 0.42515395 0.25470622 0.02791592]


In [5]:
P = dykstra_coupling(Q, nu, mu, verbose=False, max_iter=10000, tol=1e-10)

print("Projected P:")
print(P)

print("\nRow sums of P:", P.sum(axis=1))

print("Target row sums (nu):", nu)
print("\nDifference in row sums:", P.sum(axis=1) - nu)

print("\nColumn sums of P:", P.sum(axis=0))
print("\nDifference in column sums:", P.sum(axis=0) - mu)

Projected P:
[[0.16076328 0.13517202 0.         0.        ]
 [0.13146063 0.         0.         0.        ]
 [0.         0.28998194 0.25470622 0.02791592]]

Row sums of P: [0.2959353  0.13146063 0.57260407]
Target row sums (nu): [0.2959353  0.13146063 0.57260407]

Difference in row sums: [ 1.19423693e-10  2.51845822e-10 -6.99540426e-11]

Column sums of P: [0.29222391 0.42515395 0.25470622 0.02791592]

Difference in column sums: [-6.99538205e-11  6.21613316e-11  1.54554591e-10  1.54553263e-10]


Now we takle the $Z$ projection

In [11]:
def frobennius_ball_proj(Q, tau):
    norm_Q = np.linalg.norm(Q, ord='fro')
    if norm_Q <= tau:
        return Q
    else:
        return (tau / norm_Q) * Q

In [7]:
print("Frobenius norm of Q:")
print(np.linalg.norm(Q, ord='fro'))
proj= frobennius_ball_proj(Q, 15)
print("Frobenius norm of projected Q:")
print(np.linalg.norm(proj, ord='fro'))  

Frobenius norm of Q:
54.8401567377503
Frobenius norm of projected Q:
15.000000000000002


Now lets prove our algoritm with a "random" matrix

In [24]:
#Dimentions of our matrix Q
n,m = np.random.randint(3,10), np.random.randint(3,10)
#Random matrix Q and radius tau
Q = np.random.uniform(-10,10,(n,m))
tau = np.random.uniform(5,15)

#Random variables mu an nu
mu = np.random.dirichlet(np.ones(m))
nu = np.random.dirichlet(np.ones(n))

In [25]:
print(f"Matrix Q:\n{Q}")
print(f"Frobenius norm of Q: {np.linalg.norm(Q, ord='fro')}\n")
print(f"Vector nu:\n{nu}")
print(f"Vector mu:\n{mu}")
print(f"Scalar tau:\n{tau}")

Matrix Q:
[[-8.81891451 -6.37541128  6.45944639 -6.17378659  0.7672133  -9.90684242
   7.40641844]
 [-0.44982799 -3.72675105 -5.33748406  6.29167305 -2.40575704  5.86990324
   5.70539417]
 [ 5.72825188  8.00504984 -2.26213947 -2.92827986  1.57895424 -0.1684029
  -2.36792087]
 [ 8.87269101  9.30473141 -1.33363908 -7.77401413 -1.20959235 -4.92136785
  -7.39976006]
 [-6.2032275   2.20752057 -8.60955678  1.64193424  0.0621944  -1.43890914
   2.97994927]
 [-9.00349697  8.44933844 -0.45139588  6.12507485 -0.62123997 -0.12430876
   5.12874294]]
Frobenius norm of Q: 35.79785636381666

Vector nu:
[0.25328964 0.01216681 0.18613003 0.03969973 0.43783353 0.07088025]
Vector mu:
[0.27268444 0.11379236 0.14649463 0.06773576 0.17003506 0.16895428
 0.06030346]
Scalar tau:
10.151858079261265


In [12]:
def admm_transport_projection(
    pi,
    nu,
    mu,
    tau,
    rho=1.0,
    max_iter=200,
    tol=1e-6,
    dykstra_iters=10,
    verbose=False
):

    P = pi.copy()
    Z = pi.copy()
    U = np.zeros_like(pi)

    for k in range(max_iter):

        P_old = P.copy()

        # ---- P update ----
        M = (pi + rho * (Z - U)) / (1.0 + rho)
        P = dykstra_coupling(
            M, nu, mu,
            max_iter=dykstra_iters
        )

        # ---- Z update ----
        Z = frobennius_ball_proj(P + U, tau)

        # ---- dual update ----
        U = U + P - Z

        # ---- stopping ----
        r_norm = np.linalg.norm(P - Z, ord='fro')
        s_norm = np.linalg.norm(P - P_old, ord='fro')

        if verbose and k % 20 == 0:
            print(
                f"Iter {k:4d} | "
                f"primal = {r_norm:.2e} | "
                f"dual = {s_norm:.2e}"
            )

        if r_norm < tol and s_norm < tol:
            if verbose:
                print(f"Convergió en {k} iteraciones")
            break

    return P, Z


In [45]:
P,Z = admm_transport_projection(
    Q, nu, mu, tau, rho = 1.0, verbose=False, max_iter=500000, tol=1e-8, dykstra_iters=100000)

In [46]:
print("Projected P:\n")
print(P)
print(f"Frobennius norm of P: {np.linalg.norm(P, ord='fro')}\n")
print(f"Row sum of P diference: {P.sum(axis=1) - nu}\n")
print(f"Column sum of P diference: {P.sum(axis=0) - mu}\n")

Projected P:

[[0.         0.         0.16272106 0.         0.         0.
  0.07652989]
 [0.         0.         0.         0.         0.         0.01471944
  0.        ]
 [0.20399681 0.         0.         0.         0.         0.
  0.        ]
 [0.05756651 0.         0.         0.         0.         0.
  0.        ]
 [0.         0.04236469 0.         0.06974103 0.17204034 0.15624012
  0.        ]
 [0.         0.07343292 0.         0.         0.         0.
  0.        ]]
Frobennius norm of P: 0.37886680420331936

Row sum of P diference: [-0.01403869  0.00255263  0.01786678  0.01786678  0.00255265  0.00255268]

Column sum of P diference: [-0.01112112  0.00200526  0.01622643  0.00200527  0.00200527  0.00200528
  0.01622643]



## Proximal Gradient + barrier (Converges only on the frobenius norm)

Remember taht our problem is:

$$
\begin{dcases}
    \min_{\mathcal{B}_\tau} \frac{1}{2} ||P - \pi ||_F^2\\

    \textbf{s.t }\sum_{r=1}^R P_{rs}^{(m)} = q_s^{(m)}, \quad 
    \sum_{s=1}^{S^{(m)}} P_{rs}^{(m)} = p_r
\end{dcases}
$$

We can relax this probem by setting:

$$
\min \frac{1}{2} ||P - \pi ||_F^2 + \frac{\lambda}{2} ||P1-\nu||_2^2 + \frac{\beta}{2} ||P^T 1- \mu||_2^2\\

\textbf{s.t } P \geq 0, P \in \mathcal{B}_\tau
$$

Lets calculate the gradient of the prevuios function, lets call it $F(P)$:

$$
\nabla F(P) =P - \pi + \lambda(P1-\nu)1^T + \beta 1(P^T 1 - \mu)^T
$$

So the algo would look like:

$$
\bar{P}^{k+1} = P^k - \xi\nabla F(P^k)\\
P^{k+1} = \mathcal{P}_{\mathcal{B}_\tau}(P^{k,+})
$$

Lets prove our algo:

In [47]:
#Dimentions of our matrix Q
n,m = np.random.randint(3,10), np.random.randint(3,10)
#Random matrix Q and radius tau
Q = np.random.uniform(-10,10,(n,m))
tau = np.random.uniform(5,15)

#Random variables mu an nu
mu = np.random.dirichlet(np.ones(m))
nu = np.random.dirichlet(np.ones(n))

print(f"Matrix Q:\n{Q}")
print(f"Frobenius norm of Q: {np.linalg.norm(Q, ord='fro')}\n")
print(f"Vector nu:\n{nu}")
print(f"Vector mu:\n{mu}")
print(f"Scalar tau:\n{tau}")

Matrix Q:
[[ 6.94617972 -6.23651455  0.73751336  5.8402345   8.55951826 -6.93005099
  -3.28438114  2.50769877]
 [ 8.57916458  5.6076939  -0.2756286   8.08306244  6.5484072   7.02559511
   8.07498024  4.01999045]
 [ 0.15798905 -3.64294244 -9.97974674  4.65856153  0.11174263  4.30463063
   5.98020255 -8.52134627]
 [ 8.29108061  2.50682074  8.49956537 -6.16491754 -9.99326984 -5.33087101
  -7.15262262  7.66869989]
 [-0.18109     4.3970061   6.51999685 -5.30664642 -0.77182969  8.17298109
  -9.92035715  1.7379725 ]
 [ 1.20359372 -2.52256672 -5.96712583  5.83061841  9.51642489 -2.5035926
   0.72064171  2.156774  ]
 [ 5.79501081  9.50154427 -3.91916999  7.91948883  0.46834937  1.53072088
   4.2197532  -2.83974769]
 [ 8.11199148 -1.11963896 -8.146214    9.37907912  5.974018    0.34361907
   6.46851926  1.64537556]]
Frobenius norm of Q: 47.53592012813715

Vector nu:
[0.11861878 0.17602565 0.01947676 0.1823152  0.0217832  0.15210973
 0.23961732 0.09005336]
Vector mu:
[0.04413    0.40818031 0.0355

In [49]:
def proximal_gradient_transport_projection(
    pi,
    nu,
    mu,
    tau,
    lambda_=1.0,
    beta = 1.0,
    step_size=1.0,
    max_iter=20000,
    tol=1e-8,
    verbose=False
):
    P = pi.copy()

    for k in range(max_iter):

        P_old = P.copy()

        firstpart =  (P - pi) 
        secondpart =  lambda_ * (P.sum(axis=1) - nu)[:, None]
        thirdpart =  beta * (P.sum(axis=0) - mu)[None, :]

        gradient = firstpart + secondpart + thirdpart

        P -= step_size * gradient

        # ---- Projection step ----
        P = proj_simplex(P)
        
        P = frobennius_ball_proj(P, tau)

        # ---- Stopping criteria ----
        rel_err = (
            np.linalg.norm(P - P_old, ord='fro')
            / (np.linalg.norm(P_old, ord='fro') + 1e-12)
        )

        if verbose and k % 50 == 0:
            marg_r = np.linalg.norm(P.sum(axis=1) - nu)
            marg_c = np.linalg.norm(P.sum(axis=0) - mu)
            print(
                f"Iter {k:4d} | "
                f"rel = {rel_err:.2e} | "
                f"rows = {marg_r:.2e} | "
                f"cols = {marg_c:.2e}"
            )

        if rel_err < tol:
            if verbose:
                print(f"Convergió en {k} iteraciones")
            break

    return P

In [58]:
P = proximal_gradient_transport_projection(
    Q, nu, mu, tau, lambda_=1e10, beta=1000.0,)

print("Projected P:")
print(P)

print(f"Frobennius norm of P: {np.linalg.norm(P, ord='fro')}\n")
print(f"Row sum of P diference: {P.sum(axis=1) - nu}\n")
print(f"Column sum of P diference: {P.sum(axis=0) - mu}\n")



Projected P:
[[0.71116754 0.71116775 0.71116753 0.71116752 0.71116752 0.71116752
  0.71116766 0.71116764]
 [1.05534558 1.05534579 1.05534557 1.05534556 1.05534555 1.05534557
  1.0553457  1.05534567]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.13059795 0.13059817 0.13059795 0.13059793 0.13059793 0.13059795
  0.13059807 0.13059805]
 [0.91195966 0.91195988 0.91195965 0.91195965 0.91195964 0.91195965
  0.91195978 0.91195976]
 [1.43660411 1.43660433 1.4366041  1.4366041  1.43660408 1.4366041
  1.43660423 1.4366042 ]
 [0.53990593 0.53990614 0.53990591 0.53990591 0.53990591 0.53990592
  0.53990605 0.53990602]]
Frobennius norm of P: 6.211969114558373

Row sum of P diference: [ 5.5707219   8.26673934 -0.01947676 -0.1823152   1.02300081  7.14356794
 11.25321593  4.22919444]

Column sum of P diference: [4.74145077 4.37740176 4.75002215 4.76898019 4.77847619 4.75521964


## How does MAM works?

We are given $m$ measures with support:

$$
\text{Supp}(\nu^{(m)}) = \{\zeta^{m}_{S^{i}}\}_{i = 1,...,S^m}
$$

The $m$-th measure has $S^{m}$ elements on its support. And the goal measure has $R$ points. We also have: $\nu^{m} = \sum_{s = 1}^{S^m} q_s^{(m)} \delta_{\zeta_s^{(m)}}$ that is: $q_s^{(m)}$ is the mass on the point $\zeta_s^{(m)}$. Taking this into account lets look at the MAM algo:


**Step 0: input**

1: Given $\rho > 0$, the cost matrix and initial point $c, \theta^0 \in \mathbb{R}^{R \times T}$, and $a \in \Delta_M$ as in (5.2a), set
$$k \leftarrow 0 \text{ and } p_r^{(m),0} \leftarrow \sum_{s=1}^{S^{(m)}} \theta_{rs}^{(m),0}, \quad r=1,\ldots,R, \quad m=1,\ldots,M$$

2: Set $\gamma \leftarrow \infty$ if $q^{(m)} \in \mathbb{R}_+^{S^{(m)}}$, $m=1,\ldots,M$, are balanced; otherwise, choose $\gamma \in [0,\infty)$

3: **while** not converged **do**

**Step 1: average the marginals**

4: Compute 

$$
p^k \leftarrow \sum_{m=1}^M a_m p^{(m)}
$$

5: Set $l^k = 1$ if $\rho \sqrt{\sum_{m=1}^M \frac{\|p^k - p^{(m)}\|^2}{S^{(m)}}} \leq \gamma$; otherwise, $l^k \leftarrow \gamma / \left(\rho \sqrt{\sum_{m=1}^M \frac{\|p^k - p^{(m)}\|^2}{S^{(m)}}}\right)$

6: Choose an index set $\emptyset \neq \mathcal{M}^k \subseteq \{1,\ldots,M\}$

7: **for** $m \in \mathcal{M}^k$ **do**

**Step 2: update the $m^{th}$ plan**

8: **for** $s = 1,\ldots,S^{(m)}$ **do**

9: Define $w_r \leftarrow \theta_{rs}^{(m),k} + 2l^k \frac{p_r^k - p_r^{(m)}}{S^{(m)}} - \frac{1}{\rho} c_{rs}^{(m)}, \quad r=1,\ldots,R$,

10: Compute $(\hat{\pi}_1^{(m)}, \ldots, \hat{\pi}_{RS}^{(m)}) \leftarrow \text{Proj}_{\Delta_R} q_s^{(m)} (w)$

11: Update $\theta_{rs}^{(m),k+1} \leftarrow \hat{\pi}_{rs}^{(m)} - l^k \frac{p_r^k - p_r^{(m)}}{S^{(m)}}, \quad r=1,\ldots,R$,

12: **end for**

**Step 3: update the $m^{th}$ marginal**

13: Update $p_r^{(m)} \leftarrow \sum_{s=1}^{S^{(m)}} \theta_{rs}^{(m),k+1}, \quad r=1,\ldots,R$

14: **end for**

15: **end while**

16: Return $\hat{p} \leftarrow p^k$

The inputs are a cost matrix $c$ and a intial set of plans: $\theta^0 \in \mathbb{R}^{R \times \sum_{m=1}^M s^m}$ and the marginals $p^m$. next we may average the marginals: 
$$
p^k = \sum_{m=1}^M a_m p^{(m)}
$$ 
And choose a $t^k$. For the update of the plans we do:
1. Choose a set of indeces (to work with the corresponding plans)
2. For each plan $m$ on the indeces, and each column $s = 1,...,S^m$ on the plan:
   - Define $w_r$ for each row of the plan ($r=1,...,R$) (So we get, for all of the plan a total of $R*S(m)$ vaues for $w_r$), this leads to a vector $w$ for each column
   - Do: $\text{proj}_{\Delta_R (q_s^{(m)})}(w) = (\pi_{1s}^m,...,\pi_{Rs}^m)$, that is, you project $w$ on the scaled simplex to get a "new" version of the column $s$ of the $m$-th "plan"
   - You follow the uploads: $\theta_{rs}^{(m),k+1} \leftarrow \hat{\pi}_{rs}^{(m)} - l^k \frac{p_r^k - p_r^{(m)}}{S^{(m)}}, \quad r=1,\ldots,R$, that is the, as you hace a fixed plan, and a fixed column from that plan, you upload the elements of that column following the previous upload.
   - After that you upload the marginals:$p_r^{(m)} \leftarrow \sum_{s=1}^{S^{(m)}} \theta_{rs}^{(m),k+1}, \quad r=1,\ldots,R$

At the end of the inner for (the one who runs on the columns) you get the plan uploated, __here we could implement our previos code (We should see if we do it before or after uloading the marginals)__. After that, at the end of the outer for, you get the plans of $\mathcal{M}$ uploaded